对数据集进行分组并对各组应用一个函数（无论是聚合还是转换），通常是数据分析工作中的重要环节。在将数据集加载、融合、准备好之后，通常就是计算分组统计或生成透视表。

在本章中，你将会学到：

> * 使用一个或多个键（形式可以是函数、数组或DataFrame列名）分割pandas对象。
> * 计算分组的概述统计，比如数量、平均值或标准差，或是用户定义的函数。
> * 应用组内转换或其他运算，如规格化、线性回归、排名或选取子集等。
> * 计算透视表或交叉表。
> * 执行分位数分析以及其它统计分组分析。


# 10.1 GroupBy机制 GroupBy Mechanics-----------------------------------------------------

Hadley Wickham（许多热门R语言包的作者）创造了一个用于表示分组运算的术语"split-apply-combine"（拆分－应用－合并）。

第一个阶段，*pandas对象（无论是Series、DataFrame还是其他的）中的数据*会根据你所提供的一个或多个键被拆分（split）为多组。拆分操作是在对象的特定轴上执行的。例如，DataFrame可以在其行（axis=0）或列（axis=1）上进行分组。然后，将一个函数应用（apply）到各个分组并产生一个新值。最后，所有这些函数的执行结果会被合并（combine）到最终的结果对象中。

分组键可以有多种形式，且类型不必相同：注意，后三种都只是快捷方式而已，其最终目的仍然是产生一组用于拆分对象的值。

> * 列表或数组，其长度与待分组的轴一样。
> * 表示DataFrame某个列名的值。
> * 字典或Series，给出待分组轴上的值与分组名之间的对应关系。
> * 函数，用于处理轴索引或索引中的各个标签。

In [1]:
import pandas as pd
import numpy as np

df = pd.DataFrame({'key1' : ['a', 'a', 'b', 'b', 'a'],
                   'key2' : ['one', 'two', 'one', 'two', 'one'],
                   'data1' : np.random.randn(5),
                   'data2' : np.random.randn(5)})
df

,data1,data2,key1,key2
0,-1.339479,1.365165,a,one
1,-2.380970,-0.344652,a,two
2,-1.834979,0.095645,b,one
3,-0.124940,-0.119398,b,two
4,-0.155111,-1.481929,a,one


假设你想要按key1进行分组，并计算data1列的平均值。实现该功能的方式有很多，而我们这里要用的是：访问data1，并根据key1调用groupby：

变量grouped是一个**GroupBy对象**。它实际上还没有进行任何计算，只是含有一些有关分组键df['key1']的中间数据而已。换句话说，该对象已经有了接下来对各分组执行运算所需的一切信息。例如，我们可以调用GroupBy的mean方法来计算分组平均值：

稍后我将详细讲解.mean()的调用过程。这里最重要的是，数据（Series）根据分组键进行了聚合，产生了一个新的Series，其索引为key1列中的唯一值。之所以结果中索引的名称为key1，是因为原始DataFrame的列df['key1']就叫这个名字。

In [2]:
grouped = df['data1'].groupby(df['key1'])
grouped

In [3]:
grouped.mean()

key1
a   -1.291853
b   -0.979959
Name: data1, dtype: float64

如果我们一次传入多个数组的列表，就会得到不同的结果：

这里，我通过两个键对数据进行了分组，得到的Series具有一个层次化索引（由唯一的键对组成）：【根据多个键分组，会形成层次化索引】

在这个例子中，分组键均为Series。实际上，分组键可以是任何长度适当的数组：【**长度对应才能正确对应数据**】

通常，分组信息就位于相同的要处理DataFrame中。这里，你还可以将列名（可以是字符串、数字或其他Python对象）用作分组键：

你可能已经注意到了，第一个例子在执行df.groupby('key1').mean()时，结果中没有key2列。这是因为df['key2']不是数值数据（俗称“麻烦列”），所以被从结果中排除了。默认情况下，所有数值列都会被聚合，虽然有时可能会被过滤为一个子集，稍后就会碰到。【**非数据列默认从结果中排除**】

无论你准备拿groupby做什么，都有可能会用到GroupBy的**size方法**，它可以返回一个含有分组大小的Series：

In [4]:
means = df['data1'].groupby([df['key1'],df['key2']]).mean()
means

key1  key2
a     one    -0.747295
      two    -2.380970
b     one    -1.834979
      two    -0.124940
Name: data1, dtype: float64

In [5]:
means.unstack()

key2,one,two
key1,,
a,-0.747295,-2.38097
b,-1.834979,-0.12494


In [6]:
states = np.array(['Ohio', 'California', 'California', 'Ohio', 'Ohio'])
year = np.array([2005, 2005, 2006, 2005, 2006])

df['data1'].groupby([states, year]).mean()

California  2005   -2.380970
            2006   -1.834979
Ohio        2005   -0.732209
            2006   -0.155111
Name: data1, dtype: float64

In [7]:
df.groupby('key1').mean()

,data1,data2
key1,,
a,-1.291853,-0.153806
b,-0.979959,-0.011877


In [8]:
df.groupby(['key1','key2']).mean()

data1     data2
key1 key2                    
a    one  -0.747295 -0.058382
     two  -2.380970 -0.344652
b    one  -1.834979  0.095645
     two  -0.124940 -0.119398

In [9]:
df.groupby(['key1', 'key2']).size()

key1  key2
a     one     2
      two     1
b     one     1
      two     1
dtype: int64

In [10]:
df['data1'].groupby([states, year]).size()

California  2005    1
            2006    1
Ohio        2005    2
            2006    1
Name: data1, dtype: int64

# 对分组进行迭代 Iterating Over Groups

GroupBy对象支持迭代，可以产生一组二元元组（由**分组名**和**数据块**组成）。看下面的例子：

【数据块也是DataFrame类型，包含了原始数据的索引、行、列等信息】

对于多重键的情况，元组的第一个元素将会是由键值组成的元组：

当然，你可以对这些数据片段做任何操作。有一个你可能会觉得有用的运算：**将这些数据片段做成一个字典**：【通过键值对的方式来取出来】

groupby默认是在axis=0上进行分组的，通过设置也可以在其他任何轴上进行分组。拿上面例子中的df来说，我们可以**根据dtype对列进行分组**：

In [11]:
for name,group in df.groupby('key1'):
    print(name)
    print(group)

a
      data1     data2 key1 key2
0 -1.339479  1.365165    a  one
1 -2.380970 -0.344652    a  two
4 -0.155111 -1.481929    a  one
b
      data1     data2 key1 key2
2 -1.834979  0.095645    b  one
3 -0.124940 -0.119398    b  two


In [12]:
 for (k1, k2), group in df.groupby(['key1', 'key2']):
        print((k1, k2))
        print(group)

('a', 'one')
      data1     data2 key1 key2
0 -1.339479  1.365165    a  one
4 -0.155111 -1.481929    a  one
('a', 'two')
     data1     data2 key1 key2
1 -2.38097 -0.344652    a  two
('b', 'one')
      data1     data2 key1 key2
2 -1.834979  0.095645    b  one
('b', 'two')
     data1     data2 key1 key2
3 -0.12494 -0.119398    b  two


In [13]:
 for (k1, k2), group in df.groupby([states, year]):
        print((k1, k2))
        print(group)

('California', 2005)
     data1     data2 key1 key2
1 -2.38097 -0.344652    a  two
('California', 2006)
      data1     data2 key1 key2
2 -1.834979  0.095645    b  one
('Ohio', 2005)
      data1     data2 key1 key2
0 -1.339479  1.365165    a  one
3 -0.124940 -0.119398    b  two
('Ohio', 2006)
      data1     data2 key1 key2
4 -0.155111 -1.481929    a  one


In [14]:
pieces = dict(list(df.groupby('key1')))
print(pieces.keys())
print(pieces.values())
print(type(pieces['b']))
pieces['b']

dict_keys(['a', 'b'])
dict_values([      data1     data2 key1 key2
0 -1.339479  1.365165    a  one
1 -2.380970 -0.344652    a  two
4 -0.155111 -1.481929    a  one,       data1     data2 key1 key2
2 -1.834979  0.095645    b  one
3 -0.124940 -0.119398    b  two])
<class 'pandas.core.frame.DataFrame'>


,data1,data2,key1,key2
2,-1.834979,0.095645,b,one
3,-0.124940,-0.119398,b,two


In [15]:
grouped = df.groupby(df.dtypes, axis=1)
print(grouped.size())
for dtype, group in grouped:
    print(dtype)
    print(group)

float64    2
object     2
dtype: int64
float64
      data1     data2
0 -1.339479  1.365165
1 -2.380970 -0.344652
2 -1.834979  0.095645
3 -0.124940 -0.119398
4 -0.155111 -1.481929
object
  key1 key2
0    a  one
1    a  two
2    b  one
3    b  two
4    a  one


# 选取一列或列的子集 Selecting a Column or Subset of Columns

对于由DataFrame产生的GroupBy对象，如果用一个（单个字符串）或一组（字符串数组）列名对其进行索引，就能实现选取部分列进行聚合的目的。也就是说：
```
df.groupby('key1')['data1']
df.groupby('key1')[['data2']]
```
是以下代码的语法糖：
```
df['data1'].groupby(df['key1'])
df[['data2']].groupby(df['key1'])
```

尤其对于大数据集，很可能只需要对部分列进行聚合。例如，在前面那个数据集中，如果只需计算data2列的平均值并以DataFrame形式得到结果，可以这样写：

这种**索引操作**所返回的对象是一个已分组的DataFrame（如果传入的是列表或数组）或已分组的Series（如果传入的是标量形式的单个列名）：

In [16]:
df.groupby(['key1', 'key2'])[['data2']].mean()

data2
key1 key2          
a    one  -0.058382
     two  -0.344652
b    one   0.095645
     two  -0.119398

In [17]:
s_grouped = df.groupby(['key1', 'key2'])['data2']
s_grouped.mean()

key1  key2
a     one    -0.058382
      two    -0.344652
b     one     0.095645
      two    -0.119398
Name: data2, dtype: float64

# 通过字典或Series进行分组 Grouping with Dicts and Series

除数组以外，分组信息还可以其他形式存在。来看另一个示例DataFrame：

现在，假设已知列的分组关系，并希望根据分组计算列的和：

现在，你可以将这个**字典传给groupby**，来构造数组，但我们可以直接传递字典（我包含了键“f”来强调，存在未使用的分组键是可以的）：

传入Series类型，也有同样的功能，它可以被看做一个固定大小的映射：

In [18]:
people = pd.DataFrame(np.random.randn(5, 5),
   ....:                       columns=['a', 'b', 'c', 'd', 'e'],
   ....:                       index=['Joe', 'Steve', 'Wes', 'Jim', 'Travis'])
people.iloc[2:3, [1,2]] = np.nan
people

,a,b,c,d,e
Joe,-0.208966,1.681864,1.203871,-1.098336,0.724069
Steve,-0.683068,-0.461543,-0.285727,-1.927123,-1.096855
Wes,-1.005923,NaN,NaN,1.570237,1.584579
Jim,0.160217,-0.762854,0.580419,-0.101684,-1.203621
Travis,-2.417696,0.439228,1.539457,0.070633,0.906780


In [19]:
mapping = {'a': 'red', 'b': 'red', 'c': 'blue',
           'd': 'blue', 'e': 'red', 'f' : 'orange'}
by_column = people.groupby(mapping, axis=1)
by_column.sum()
# dict(list(by_column))    # 此方式查看下原始数据具体怎么groupby的

,blue,red
Joe,0.105535,2.196968
Steve,-2.212850,-2.241465
Wes,1.570237,0.578655
Jim,0.478735,-1.806258
Travis,1.610090,-1.071687


In [20]:
map_series = pd.Series(mapping)
people.groupby(map_series, axis=1).count()

,blue,red
Joe,2,3
Steve,2,3
Wes,1,2
Jim,2,3
Travis,2,3


# 通过函数进行分组 Grouping with Functions

比起使用字典或Series，使用Python函数是一种更原生的方法定义分组映射。任何**被当做分组键的函数**都会在**各个索引值**上被调用一次，其返回值就会被用作分组名称。【注意：有点类似字典的映射关系，要小心此函数是否能处理的了索引值。】

具体点说，以上一小节的示例DataFrame为例，其索引值为人的名字。你可以计算一个字符串长度的数组，更简单的方法是传入len函数：



将函数跟数组、列表、字典、Series**混合使用**也不是问题，因为任何东西在内部都会被转换为数组：

In [21]:
# people.groupby(len).sum()
dict(list(people.groupby(len)))

{3:             a         b         c         d         e
 Joe -0.208966  1.681864  1.203871 -1.098336  0.724069
 Wes -1.005923       NaN       NaN  1.570237  1.584579
 Jim  0.160217 -0.762854  0.580419 -0.101684 -1.203621,
 5:               a         b         c         d         e
 Steve -0.683068 -0.461543 -0.285727 -1.927123 -1.096855,
 6:                a         b         c         d        e
 Travis -2.417696  0.439228  1.539457  0.070633  0.90678}

In [22]:
df.groupby(lambda x:'even' if x%2==0 else 'odd').mean()

,data1,data2
even,-1.109856,-0.007040
odd,-1.252955,-0.232025


In [23]:
key_list = ['one', 'one', 'one', 'two', 'two']
people.groupby([len, key_list]).min()

a         b         c         d         e
3 one -1.005923  1.681864  1.203871 -1.098336  0.724069
  two  0.160217 -0.762854  0.580419 -0.101684 -1.203621
5 one -0.683068 -0.461543 -0.285727 -1.927123 -1.096855
6 two -2.417696  0.439228  1.539457  0.070633  0.906780

# 根据索引级别分组 Grouping by Index Levels

层次化索引数据集最方便的地方就在于它能够根据轴索引的一个级别进行聚合：

要根据级别分组，使用**level关键字**传递级别序号或名字：【注意axis要指定为对应的，才能符合逻辑】

In [24]:
columns = pd.MultiIndex.from_arrays([['US', 'US', 'US', 'JP', 'JP'],
   ....:                                     [1, 3, 5, 1, 3]],
   ....:                                     names=['cty', 'tenor'])
hier_df = pd.DataFrame(np.random.randn(4, 5), columns=columns)
hier_df

cty          US                            JP          
tenor         1         3         5         1         3
0     -0.719884  0.867574  1.763689  1.213179  0.242836
1     -2.119959  1.415971  1.699411  0.330929  0.329141
2     -1.184357  0.976098 -0.453770  0.149631  0.678433
3      0.717542 -1.170957  1.450431 -0.496822  0.158387

In [25]:
hier_df.groupby(level='cty', axis=1).count()
dict(list(hier_df.groupby(level='cty', axis=1)))

{'JP': cty          JP          
 tenor         1         3
 0      1.213179  0.242836
 1      0.330929  0.329141
 2      0.149631  0.678433
 3     -0.496822  0.158387, 'US': cty          US                    
 tenor         1         3         5
 0     -0.719884  0.867574  1.763689
 1     -2.119959  1.415971  1.699411
 2     -1.184357  0.976098 -0.453770
 3      0.717542 -1.170957  1.450431}

# 10.2 数据聚合 Data Aggregation---------------------------------------------------------------

*聚合指的是任何能够从数组产生标量值的数据转换过程。*

之前的例子已经用过一些，比如mean、count、min以及sum等。你可能想知道在GroupBy对象上调用mean()时究竟发生了什么。许多**常见的聚合运算**（如表10-1所示）都有进行优化。然而，除了这些方法，你还可以使用其它的。

> 表10-1 常见聚合运算

> count | sum | mean | median | std var | min max | prod非NA值的积 | first last第一或最后一个非NA值 |

你可以使用自己发明的聚合运算，还可以**调用分组对象上已经定义好的任何方法**。例如，quantile可以计算Series或DataFrame列的样本分位数。

虽然quantile并没有明确地实现于GroupBy，但它是一个Series方法，所以这里是能用的。实际上，GroupBy会高效地对Series进行切片，然后**对各片调用聚合方法**piece.quantile(0.9)，最后将这些结果组装成最终结果：

如果要使用你自己的聚合函数，只需**将其传入aggregate或agg方法**即可：

你可能注意到注意，有些方法（如describe）也是可以用在这里的，即使严格来讲，它们并非聚合运算：

笔记：自定义聚合函数要比表10-1中那些经过优化的函数慢得多。这是因为在构造中间分组数据块时存在非常大的开销（函数调用、数据重排等）。

In [26]:
df

,data1,data2,key1,key2
0,-1.339479,1.365165,a,one
1,-2.380970,-0.344652,a,two
2,-1.834979,0.095645,b,one
3,-0.124940,-0.119398,b,two
4,-0.155111,-1.481929,a,one


In [27]:
grouped = df.groupby('key1')
dict(list(grouped))

{'a':       data1     data2 key1 key2
 0 -1.339479  1.365165    a  one
 1 -2.380970 -0.344652    a  two
 4 -0.155111 -1.481929    a  one, 'b':       data1     data2 key1 key2
 2 -1.834979  0.095645    b  one
 3 -0.124940 -0.119398    b  two}

In [28]:
grouped['data1'].quantile(0.9)

key1
a   -0.391984
b   -0.295944
Name: data1, dtype: float64

In [29]:
def peak_to_peak(arr):
    return arr.max()-arr.min()    # arr.iloc[0]

grouped.agg(peak_to_peak)

,data1,data2
key1,,
a,2.225859,2.847094
b,1.710039,0.215043


In [30]:
grouped.describe()

data1                                                              \
     count      mean       std       min       25%       50%       75%   
key1                                                                     
a      3.0 -1.291853  1.113694 -2.380970 -1.860224 -1.339479 -0.747295   
b      2.0 -0.979959  1.209180 -1.834979 -1.407469 -0.979959 -0.552450   

               data2                                                    \
           max count      mean       std       min       25%       50%   
key1                                                                     
a    -0.155111   3.0 -0.153806  1.433109 -1.481929 -0.913291 -0.344652   
b    -0.124940   2.0 -0.011877  0.152058 -0.119398 -0.065637 -0.011877   

                          
           75%       max  
key1                      
a     0.510256  1.365165  
b     0.041884  0.095645

# 面向列的多函数应用 Column-Wise and Multiple Function Application

回到前面小费的例子。使用read_csv导入数据之后，我们添加了一个小费百分比的列tip_pct：

你已经看到，对Series或DataFrame列的聚合运算其实就是使用aggregate（使用自定义函数）或调用诸如mean、std之类的方法。然而，你可能希对不同的列使用不同的聚合函数，或一次应用多个函数。其实这也好办，我将通过一些示例来进行讲解。首先，我根据天和smoker对tips进行分组：

注意，对于表10-1中的那些描述统计，可以将**函数名以字符串的形式传入agg函数**：

如果**传入一组函数或函数名**，得到的DataFrame的列就会以相应的函数命名：

这里，我们传递了一组聚合函数进行聚合，独立对数据分组进行评估。

In [31]:
tips = pd.read_csv('examples/tips.csv')
tips['tip_pct'] = tips['tip'] / tips['total_bill']
tips.head()

,total_bill,tip,smoker,day,time,size,tip_pct
0,16.99,1.01,No,Sun,Dinner,2,0.059447
1,10.34,1.66,No,Sun,Dinner,3,0.160542
2,21.01,3.50,No,Sun,Dinner,3,0.166587
3,23.68,3.31,No,Sun,Dinner,2,0.139780
4,24.59,3.61,No,Sun,Dinner,4,0.146808


In [32]:
grouped = tips.groupby(['day', 'smoker'])
grouped_pct = grouped['tip_pct']
grouped_pct.agg('mean')

day   smoker
Fri   No        0.151650
      Yes       0.174783
Sat   No        0.158048
      Yes       0.147906
Sun   No        0.160113
      Yes       0.187250
Thur  No        0.160298
      Yes       0.163863
Name: tip_pct, dtype: float64

In [33]:
grouped_pct.agg(['mean', 'std', peak_to_peak])

mean       std  peak_to_peak
day  smoker                                  
Fri  No      0.151650  0.028123      0.067349
     Yes     0.174783  0.051293      0.159925
Sat  No      0.158048  0.039767      0.235193
     Yes     0.147906  0.061375      0.290095
Sun  No      0.160113  0.042347      0.193226
     Yes     0.187250  0.154134      0.644685
Thur No      0.160298  0.038774      0.193350
     Yes     0.163863  0.039389      0.151240

你并非一定要接受GroupBy自动给出的那些列名，特别是lambda函数，它们的名称是lambda，这样的辨识度就很低了（通过函数的name属性看看就知道了）。

因此，如果**传入(name,function)元组组成的列表**，则各元组的第一个元素就会被用作DataFrame的列名（可以将这种二元元组列表看做一个有序映射）：

对于DataFrame，你还有更多选择，你可以定义一组应用于全部列的一组函数，或不同的列应用不同的函数。假设我们想要对tip_pct和total_bill列计算三个统计信息：

如你所见，结果DataFrame拥有层次化的列，这相当于分别对各列进行聚合，然后用concat将结果组装到一起，使用列名用作keys参数：

跟前面一样，这里也可以传入带有自定义名称的一组元组：

现在，假设你想要对一个列或不同的列应用不同的函数。具体的办法是**向agg传入一个从列名映射到函数的字典**：

只有将多个函数应用到至少一列时，DataFrame才会拥有层次化的列。

In [34]:
grouped_pct.agg([('foo', 'mean'), ('bar', np.std)])

foo       bar
day  smoker                    
Fri  No      0.151650  0.028123
     Yes     0.174783  0.051293
Sat  No      0.158048  0.039767
     Yes     0.147906  0.061375
Sun  No      0.160113  0.042347
     Yes     0.187250  0.154134
Thur No      0.160298  0.038774
     Yes     0.163863  0.039389

In [35]:
functions = ['count', 'mean', 'max']
result = grouped['tip_pct', 'total_bill'].agg(functions)
result

tip_pct                     total_bill                  
              count      mean       max      count       mean    max
day  smoker                                                         
Fri  No           4  0.151650  0.187735          4  18.420000  22.75
     Yes         15  0.174783  0.263480         15  16.813333  40.17
Sat  No          45  0.158048  0.291990         45  19.661778  48.33
     Yes         42  0.147906  0.325733         42  21.276667  50.81
Sun  No          57  0.160113  0.252672         57  20.506667  48.17
     Yes         19  0.187250  0.710345         19  24.120000  45.35
Thur No          45  0.160298  0.266312         45  17.113111  41.19
     Yes         17  0.163863  0.241255         17  19.190588  43.11

In [36]:
ftuples = [('Durchschnitt', 'mean'),('Abweichung', np.var)]
grouped['tip_pct', 'total_bill'].agg(ftuples)

tip_pct              total_bill            
            Durchschnitt Abweichung Durchschnitt  Abweichung
day  smoker                                                 
Fri  No         0.151650   0.000791    18.420000   25.596333
     Yes        0.174783   0.002631    16.813333   82.562438
Sat  No         0.158048   0.001581    19.661778   79.908965
     Yes        0.147906   0.003767    21.276667  101.387535
Sun  No         0.160113   0.001793    20.506667   66.099980
     Yes        0.187250   0.023757    24.120000  109.046044
Thur No         0.160298   0.001503    17.113111   59.625081
     Yes        0.163863   0.001551    19.190588   69.808518

In [37]:
grouped.agg({ 'tip': np.max, 'size': 'sum'})

tip  size
day  smoker             
Fri  No       3.50     9
     Yes      4.73    31
Sat  No       9.00   115
     Yes     10.00   104
Sun  No       6.00   167
     Yes      6.50    49
Thur No       6.70   112
     Yes      5.00    40

In [38]:
grouped.agg({'tip_pct' : ['min', 'max', 'mean', 'std'],
             'size' : 'sum'})

tip_pct                               size
                  min       max      mean       std  sum
day  smoker                                             
Fri  No      0.120385  0.187735  0.151650  0.028123    9
     Yes     0.103555  0.263480  0.174783  0.051293   31
Sat  No      0.056797  0.291990  0.158048  0.039767  115
     Yes     0.035638  0.325733  0.147906  0.061375  104
Sun  No      0.059447  0.252672  0.160113  0.042347  167
     Yes     0.065660  0.710345  0.187250  0.154134   49
Thur No      0.072961  0.266312  0.160298  0.038774  112
     Yes     0.090014  0.241255  0.163863  0.039389   40

# 以“没有行索引”的形式返回聚合数据 Returning Aggregated Data Without Row Indexes

到目前为止，所有示例中的聚合数据都有由唯一的分组键组成的索引（可能还是层次化的）。由于并不总是需要如此，

所以你可以向groupby传入**参数as_index=False**以禁用该功能：【结果中的索引会成为列】

当然，对结果调用reset_index也能得到这种形式的结果。使用参数as_index=False可以避免一些不必要的计算。

In [39]:
tips.groupby(['day', 'smoker']).mean()

total_bill       tip      size   tip_pct
day  smoker                                          
Fri  No       18.420000  2.812500  2.250000  0.151650
     Yes      16.813333  2.714000  2.066667  0.174783
Sat  No       19.661778  3.102889  2.555556  0.158048
     Yes      21.276667  2.875476  2.476190  0.147906
Sun  No       20.506667  3.167895  2.929825  0.160113
     Yes      24.120000  3.516842  2.578947  0.187250
Thur No       17.113111  2.673778  2.488889  0.160298
     Yes      19.190588  3.030000  2.352941  0.163863

In [40]:
tips.groupby(['day', 'smoker'], as_index=False).mean()

,day,smoker,total_bill,tip,size,tip_pct
0,Fri,No,18.420000,2.812500,2.250000,0.151650
1,Fri,Yes,16.813333,2.714000,2.066667,0.174783
2,Sat,No,19.661778,3.102889,2.555556,0.158048
3,Sat,Yes,21.276667,2.875476,2.476190,0.147906
4,Sun,No,20.506667,3.167895,2.929825,0.160113
5,Sun,Yes,24.120000,3.516842,2.578947,0.187250
6,Thur,No,17.113111,2.673778,2.488889,0.160298
7,Thur,Yes,19.190588,3.030000,2.352941,0.163863


# 10.3 apply：一般性的“拆分－应用－合并”---------------------------------------------------

最通用的GroupBy方法是apply，本节剩余部分将重点讲解它。如图10-2所示，**apply方法**会将待处理的对象拆分成多个片段，然后对各片段调用传入的函数，最后尝试将各片段组合到一起。

回到之前那个小费数据集，假设你想要根据分组选出最高的5个tip_pct值。首先，编写一个选取指定列具有最大值的行的函数：

现在，如果对smoker分组并用该函数调用apply，就会得到：

这里发生了什么？top函数在DataFrame的各个片段上调用，然后结果由pandas.concat组装到一起，并以分组名称进行了标记。于是，最终结果就有了一个层次化索引，其内层索引值来自原DataFrame。

如果传给apply的函数能够接受其他参数或关键字，则可以将这些内容放在函数名后面一并传入：【**处理函数的参数传递**】

In [41]:
def top(df, n=5, column='tip_pct'):
    return df.sort_values(by=column)[-n:]

top(tips, n=6)

,total_bill,tip,smoker,day,time,size,tip_pct
109,14.31,4.00,Yes,Sat,Dinner,2,0.279525
183,23.17,6.50,Yes,Sun,Dinner,4,0.280535
232,11.61,3.39,No,Sat,Dinner,2,0.291990
67,3.07,1.00,Yes,Sat,Dinner,1,0.325733
178,9.60,4.00,Yes,Sun,Dinner,2,0.416667
172,7.25,5.15,Yes,Sun,Dinner,2,0.710345


In [42]:
tips.groupby('smoker').apply(top)

total_bill   tip smoker   day    time  size   tip_pct
smoker                                                           
No     88        24.71  5.85     No  Thur   Lunch     2  0.236746
       185       20.69  5.00     No   Sun  Dinner     5  0.241663
       51        10.29  2.60     No   Sun  Dinner     2  0.252672
       149        7.51  2.00     No  Thur   Lunch     2  0.266312
       232       11.61  3.39     No   Sat  Dinner     2  0.291990
Yes    109       14.31  4.00    Yes   Sat  Dinner     2  0.279525
       183       23.17  6.50    Yes   Sun  Dinner     4  0.280535
       67         3.07  1.00    Yes   Sat  Dinner     1  0.325733
       178        9.60  4.00    Yes   Sun  Dinner     2  0.416667
       172        7.25  5.15    Yes   Sun  Dinner     2  0.710345

In [43]:
tips.groupby(['smoker','day']).apply(top, n=1, column='total_bill')

total_bill    tip smoker   day    time  size   tip_pct
smoker day                                                             
No     Fri  94        22.75   3.25     No   Fri  Dinner     2  0.142857
       Sat  212       48.33   9.00     No   Sat  Dinner     4  0.186220
       Sun  156       48.17   5.00     No   Sun  Dinner     6  0.103799
       Thur 142       41.19   5.00     No  Thur   Lunch     5  0.121389
Yes    Fri  95        40.17   4.73    Yes   Fri  Dinner     4  0.117750
       Sat  170       50.81  10.00    Yes   Sat  Dinner     3  0.196812
       Sun  182       45.35   3.50    Yes   Sun  Dinner     3  0.077178
       Thur 197       43.11   5.00    Yes  Thur   Lunch     4  0.115982

可能你已经想起来了，之前我在GroupBy对象上调用过describe：

在GroupBy中，当你调用诸如describe之类的方法时，实际上只是应用了下面两条代码的快捷方式而已：
```
f = lambda x: x.describe()
grouped.apply(f)
```

# 禁止分组键 Suppressing the Group Keys

从上面的例子中可以看出，分组键会跟原始对象的索引共同构成结果对象中的层次化索引。将**参数group_keys=False**传入groupby即可禁止该效果：

In [44]:
tips.groupby('smoker', group_keys=False).apply(top)

,total_bill,tip,smoker,day,time,size,tip_pct
88,24.71,5.85,No,Thur,Lunch,2,0.236746
185,20.69,5.00,No,Sun,Dinner,5,0.241663
51,10.29,2.60,No,Sun,Dinner,2,0.252672
149,7.51,2.00,No,Thur,Lunch,2,0.266312
232,11.61,3.39,No,Sat,Dinner,2,0.291990
109,14.31,4.00,Yes,Sat,Dinner,2,0.279525
183,23.17,6.50,Yes,Sun,Dinner,4,0.280535
67,3.07,1.00,Yes,Sat,Dinner,1,0.325733
178,9.60,4.00,Yes,Sun,Dinner,2,0.416667
172,7.25,5.15,Yes,Sun,Dinner,2,0.710345


In [45]:
tips.groupby('smoker').apply(top)

total_bill   tip smoker   day    time  size   tip_pct
smoker                                                           
No     88        24.71  5.85     No  Thur   Lunch     2  0.236746
       185       20.69  5.00     No   Sun  Dinner     5  0.241663
       51        10.29  2.60     No   Sun  Dinner     2  0.252672
       149        7.51  2.00     No  Thur   Lunch     2  0.266312
       232       11.61  3.39     No   Sat  Dinner     2  0.291990
Yes    109       14.31  4.00    Yes   Sat  Dinner     2  0.279525
       183       23.17  6.50    Yes   Sun  Dinner     4  0.280535
       67         3.07  1.00    Yes   Sat  Dinner     1  0.325733
       178        9.60  4.00    Yes   Sun  Dinner     2  0.416667
       172        7.25  5.15    Yes   Sun  Dinner     2  0.710345

# 分位数和桶分析 Quantile and Bucket Analysis

我曾在第8章中讲过，pandas有一些能根据指定面元或样本分位数将数据拆分成多块的工具（比如cut和qcut）。

将这些函数跟groupby结合起来，就能非常轻松地实现对数据集的桶（bucket）或分位数（quantile）分析了。以下面这个简单的随机数据集为例，我们利用cut将其装入长度相等的桶中：

由cut返回的Categorical对象可直接传递到groupby。因此，我们可以像下面这样对data2列做一些统计计算：

【**Categorical对象传给groupby**】

这些都是长度相等的桶。要根据样本分位数得到大小相等的桶，使用qcut即可。传入labels=False即可只获取分位数的编号：

In [46]:
frame = pd.DataFrame({'data1': np.random.randn(1000),
                      'data2': np.random.randn(1000)})
quartiles = pd.cut(frame.data1, 4)
quartiles.head()

0    (-1.708, 0.124]
1     (0.124, 1.957]
2    (-1.708, 0.124]
3     (0.124, 1.957]
4     (0.124, 1.957]
Name: data1, dtype: category
Categories (4, interval[float64]): [(-3.548, -1.708] < (-1.708, 0.124] < (0.124, 1.957] < (1.957, 3.789]]

In [47]:
def get_stats(group):
    return { 'min': group.min(),
            'max': group.max(),
            'count': group.count(),
            'mean': group.mean()
           }

grouped = frame.data2.groupby(quartiles)
grouped.apply(get_stats)

data1                  
(-3.548, -1.708]  count     55.000000
                  max        2.034160
                  mean      -0.083245
                  min       -2.109869
(-1.708, 0.124]   count    513.000000
                  max        2.881364
                  mean      -0.030794
                  min       -3.308236
(0.124, 1.957]    count    401.000000
                  max        3.272967
                  mean       0.074205
                  min       -2.848416
(1.957, 3.789]    count     31.000000
                  max        1.602730
                  mean      -0.120933
                  min       -1.790787
Name: data2, dtype: float64

In [48]:
grouping = pd.qcut(frame.data1, 10, labels=False)
grouped = frame.data2.groupby(grouping)
grouped.apply(get_stats)

data1       
0      count    100.000000
       max        2.034160
       mean      -0.039200
       min       -2.109869
1      count    100.000000
       max        2.015252
       mean      -0.060028
       min       -2.201172
2      count    100.000000
       max        2.881364
       mean      -0.079618
       min       -3.308236
3      count    100.000000
       max        2.487190
       mean       0.027397
       min       -2.648505
4      count    100.000000
       max        2.861938
       mean       0.032476
       min       -2.185455
5      count    100.000000
       max        2.090382
       mean       0.016065
       min       -2.849304
6      count    100.000000
       max        2.989867
       mean       0.009681
       min       -2.711455
7      count    100.000000
       max        2.280561
       mean       0.139180
       min       -1.833375
8      count    100.000000
       max        2.454361
       mean       0.003011
       min       -2.848416
9      count   

# 示例：用特定于分组的值填充缺失值

对于缺失数据的清理工作，有时你会用dropna将其替换掉，而有时则可能会希望用一个固定值或由数据集本身所衍生出来的值去填充NA值。这时就得使用fillna这个工具了。在下面这个例子中，我用平均值去填充NA值：

In [49]:
s = pd.Series(np.random.randn(6))
s[::2] = np.nan
s.fillna(s.mean())

0    0.840707
1    1.545537
2    0.840707
3    1.001177
4    0.840707
5   -0.024594
dtype: float64

假设你需要对不同的分组填充不同的值。一种方法是将数据分组，并使用apply和一个能够对各数据块调用fillna的函数即可。下面是一些有关美国几个州的示例数据，这些州又被分为东部和西部：

将一些值设为缺失：

我们可以用分组平均值去填充NA值:

另外，也可以在代码中预定义各组的填充值。由于分组具有一个name属性，所以我们可以拿来用一下：

In [50]:
states = ['Ohio', 'New York', 'Vermont', 'Florida',
          'Oregon', 'Nevada', 'California', 'Idaho']
group_key = ['East']*4+['West']*4
data = pd.Series(np.random.randn(8), index=states)
data[[ 'Vermont', 'Nevada', 'Idaho']] = np.nan
data

Ohio         -0.258279
New York      0.983366
Vermont            NaN
Florida      -1.450104
Oregon        0.475762
Nevada             NaN
California    0.625324
Idaho              NaN
dtype: float64

In [51]:
data.groupby(group_key).mean()

East   -0.241672
West    0.550543
dtype: float64

In [52]:
fill_mean = lambda g: g.fillna('xxx')    # lambda g: g.fillna(g.mean())
data.groupby(group_key).apply(fill_mean)

Ohio         -0.258279
New York      0.983366
Vermont            xxx
Florida        -1.4501
Oregon        0.475762
Nevada             xxx
California    0.625324
Idaho              xxx
dtype: object

In [53]:
fill_values = { 'East': 0.5, 'West': -1}
fill_func = lambda g: g.fillna(fill_values[g.name])
data.groupby(group_key).apply(fill_func)

Ohio         -0.258279
New York      0.983366
Vermont       0.500000
Florida      -1.450104
Oregon        0.475762
Nevada       -1.000000
California    0.625324
Idaho        -1.000000
dtype: float64

# 示例：随机采样和排列

假设你想要从一个大数据集中随机抽取（进行替换或不替换）样本以进行蒙特卡罗模拟（Monte Carlo simulation）或其他分析工作。“抽取”的方式有很多，这里使用的方法是对**Series使用sample方法**：

现在我有了一个长度为52的Series，其索引包括牌名，值则是21点或其他游戏中用于计分的点数（为了简单起见，我当A的点数为1）：

现在，根据我上面所讲的，从整副牌中抽出5张，代码如下：

假设你想要从每种花色中随机抽取两张牌。由于花色是牌名的最后一个字符，所以我们可以据此进行分组，并使用apply：

In [54]:
# Hearts, Spades, Clubs, Diamonds
suits = ['H', 'S', 'C', 'D']
card_val = (list(range(1, 11)) + [10] * 3) * 4
base_names = ['A'] + list(range(2, 11)) + ['J', 'K', 'Q']
cards = []
for suit in ['H', 'S', 'C', 'D']:
    cards.extend(str(num) + suit for num in base_names)

deck = pd.Series(card_val, index=cards)
deck

AH      1
2H      2
3H      3
4H      4
5H      5
6H      6
7H      7
8H      8
9H      9
10H    10
JH     10
KH     10
QH     10
AS      1
2S      2
3S      3
4S      4
5S      5
6S      6
7S      7
8S      8
9S      9
10S    10
JS     10
KS     10
QS     10
AC      1
2C      2
3C      3
4C      4
5C      5
6C      6
7C      7
8C      8
9C      9
10C    10
JC     10
KC     10
QC     10
AD      1
2D      2
3D      3
4D      4
5D      5
6D      6
7D      7
8D      8
9D      9
10D    10
JD     10
KD     10
QD     10
dtype: int64

In [55]:
def draw(deck, n=5):
    return deck.sample(n)

get_suit = lambda card: card[-1]
deck.groupby(get_suit).apply(draw, n=2)

C  9C      9
   8C      8
D  3D      3
   10D    10
H  5H      5
   10H    10
S  9S      9
   QS     10
dtype: int64

或者，也可以这样写：

In [56]:
deck.groupby(get_suit, group_keys=False).apply(draw, n=2)

JC     10
9C      9
JD     10
5D      5
10H    10
KH     10
KS     10
JS     10
dtype: int64

# 示例：分组加权平均数和相关系数

根据groupby的“拆分－应用－合并”范式，可以进行DataFrame的列与列之间或两个Series之间的运算（比如分组加权平均）。以下面这个数据集为例，它含有分组键、值以及一些权重值：

然后可以利用category计算分组加权平均数：

In [57]:
df = pd.DataFrame({'category': ['a', 'a', 'a', 'a','b', 'b', 'b', 'b'],
                   'data': np.random.randn(8),
                   'weights': np.random.rand(8)})
df

,category,data,weights
0,a,0.298176,0.075402
1,a,0.663074,0.314664
2,a,0.589439,0.603379
3,a,1.941591,0.806461
4,b,1.457592,0.621231
5,b,0.312785,0.008325
6,b,-1.426478,0.166666
7,b,0.518649,0.903316


In [58]:
grouped = df.groupby('category')
get_wavg = lambda g: np.average(g['data'], weights=g['weights'])
grouped.apply(get_wavg)

category
a    1.195952
b    0.670102
dtype: float64

另一个例子，考虑一个来自Yahoo!Finance的数据集，其中含有几只股票和标准普尔500指数（符号SPX）的收盘价：

来做一个比较有趣的任务：计算一个由日收益率（通过百分数变化计算）与SPX之间的年度相关系数组成的DataFrame。下面是一个实现办法，我们先创建一个函数，用它计算每列和SPX列的成对相关系数：

接下来，我们使用pct_change计算close_px的百分比变化：

最后，我们用年对百分比变化进行分组，可以用一个一行的函数，从每行的标签返回每个datetime标签的year属性：

In [59]:
close_px = pd.read_csv('examples/stock_px_2.csv', parse_dates=True,
                        index_col=0)
close_px.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2214 entries, 2003-01-02 to 2011-10-14
Data columns (total 4 columns):
AAPL    2214 non-null float64
MSFT    2214 non-null float64
XOM     2214 non-null float64
SPX     2214 non-null float64
dtypes: float64(4)
memory usage: 86.5 KB


In [60]:
spx_corr = lambda x: x.corrwith(x['SPX'])
rets = close_px.pct_change().dropna()
# rets

In [61]:
get_year = lambda x: x.year
by_year = rets.groupby(get_year)
by_year.apply(spx_corr)

,AAPL,MSFT,XOM,SPX
2003,0.541124,0.745174,0.661265,1.0
2004,0.374283,0.588531,0.557742,1.0
2005,0.467540,0.562374,0.631010,1.0
2006,0.428267,0.406126,0.518514,1.0
2007,0.508118,0.658770,0.786264,1.0
2008,0.681434,0.804626,0.828303,1.0
2009,0.707103,0.654902,0.797921,1.0
2010,0.710105,0.730118,0.839057,1.0
2011,0.691931,0.800996,0.859975,1.0


当然，你还可以计算列与列之间的相关系数。这里，我们计算Apple和Microsoft的年相关系数：

In [62]:
by_year.apply(lambda g: g['AAPL'].corr(g['MSFT']))

2003    0.480868
2004    0.259024
2005    0.300093
2006    0.161735
2007    0.417738
2008    0.611901
2009    0.432738
2010    0.571946
2011    0.581987
dtype: float64

# 示例：组级别的线性回归

顺着上一个例子继续，你可以用groupby执行更为复杂的分组统计分析，只要函数返回的是pandas对象或标量值即可。例如，我可以定义下面这个regress函数（利用statsmodels计量经济学库）对各数据块执行普通最小二乘法（Ordinary Least Squares，OLS）回归：

现在，为了按年计算AAPL对SPX收益率的线性回归，执行：

In [63]:
import statsmodels.api as sm
def regress(data, yvar, xvars):
    Y = data[yvar]
    X = data[xvars]
    X['intercept'] = 1
    result = sm.OLS(Y, X).fit()
    return result.params

by_year.apply(regress, 'AAPL', ['SPX'])

D:\Anaconda2\envs\py36\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


,SPX,intercept
2003,1.195406,0.000710
2004,1.363463,0.004201
2005,1.766415,0.003246
2006,1.645496,0.000080
2007,1.198761,0.003438
2008,0.968016,-0.001110
2009,0.879103,0.002954
2010,1.052608,0.001261
2011,0.806605,0.001514


# 10.4 透视表和交叉表 Pivot Tables and Cross-Tabulation------------------------------

透视表（pivot table）是各种电子表格程序和其他数据分析软件中一种常见的数据汇总工具。它根据一个或多个键对数据进行聚合，并根据行和列上的分组键将数据分配到各个矩形区域中。

在Python和pandas中，可以通过本章所介绍的groupby功能以及（能够利用层次化索引的）重塑运算制作透视表。DataFrame有一个**pivot_table方法**，此外还有一个顶级的pandas.pivot_table函数。除能为groupby提供便利之外，pivot_table还可以添加分项小计，也叫做margins。
```
Signature: tips.pivot_table(values=None, index=None, columns=None, aggfunc='mean', fill_value=None, margins=False, dropna=True, margins_name='All')
```
回到小费数据集，假设我想要根据day和smoker计算分组平均数（pivot_table的默认聚合类型），并将day和smoker放到行上：



In [64]:
tips.head()

,total_bill,tip,smoker,day,time,size,tip_pct
0,16.99,1.01,No,Sun,Dinner,2,0.059447
1,10.34,1.66,No,Sun,Dinner,3,0.160542
2,21.01,3.50,No,Sun,Dinner,3,0.166587
3,23.68,3.31,No,Sun,Dinner,2,0.139780
4,24.59,3.61,No,Sun,Dinner,4,0.146808


In [65]:
tips.pivot_table(index=['day', 'smoker'])

size       tip   tip_pct  total_bill
day  smoker                                          
Fri  No      2.250000  2.812500  0.151650   18.420000
     Yes     2.066667  2.714000  0.174783   16.813333
Sat  No      2.555556  3.102889  0.158048   19.661778
     Yes     2.476190  2.875476  0.147906   21.276667
Sun  No      2.929825  3.167895  0.160113   20.506667
     Yes     2.578947  3.516842  0.187250   24.120000
Thur No      2.488889  2.673778  0.160298   17.113111
     Yes     2.352941  3.030000  0.163863   19.190588

可以用groupby直接来做。现在，假设我们只想聚合tip_pct和size，而且想根据time进行分组。我将smoker放到列上，把day放到行上：

In [66]:
tips.pivot_table(['tip_pct', 'size'], index=['time', 'day'])

size   tip_pct
time   day                     
Dinner Fri   2.166667  0.158916
       Sat   2.517241  0.153152
       Sun   2.842105  0.166897
       Thur  2.000000  0.159744
Lunch  Fri   2.000000  0.188765
       Thur  2.459016  0.161301

In [67]:
tips.pivot_table(['tip_pct', 'size'], index=['time', 'day'],
                 columns='smoker')

size             tip_pct          
smoker             No       Yes        No       Yes
time   day                                         
Dinner Fri   2.000000  2.222222  0.139622  0.165347
       Sat   2.555556  2.476190  0.158048  0.147906
       Sun   2.929825  2.578947  0.160113  0.187250
       Thur  2.000000       NaN  0.159744       NaN
Lunch  Fri   3.000000  1.833333  0.187735  0.188937
       Thur  2.500000  2.352941  0.160311  0.163863

还可以对这个表作进一步的处理，传入**参数margins**=True添加分项小计。这将会添加标签为All的行和列，其值对应于单个等级中所有数据的分组统计：

这里，All值为平均数：不单独考虑烟民与非烟民（All列），不单独考虑行分组两个级别中的任何单项（All行）。

要使用其他的聚合函数，将其传给**参数aggfunc**即可。例如，使用count或len可以得到有关分组大小的交叉表（计数或频率）：

如果存在空的组合（也就是NA），你可能会希望设置一个**参数fill_value**：

> pivot_table的参数说明请参见表10-2。

> values待聚合的列名称，默认聚合所有数值列 | index columns |aggfunc默认为mean | fill_value dropna margins |

In [68]:
tips.pivot_table(['tip_pct', 'size'], index=['time', 'day'],
                 columns='smoker', margins=True)

size                       tip_pct                    
smoker             No       Yes       All        No       Yes       All
time   day                                                             
Dinner Fri   2.000000  2.222222  2.166667  0.139622  0.165347  0.158916
       Sat   2.555556  2.476190  2.517241  0.158048  0.147906  0.153152
       Sun   2.929825  2.578947  2.842105  0.160113  0.187250  0.166897
       Thur  2.000000       NaN  2.000000  0.159744       NaN  0.159744
Lunch  Fri   3.000000  1.833333  2.000000  0.187735  0.188937  0.188765
       Thur  2.500000  2.352941  2.459016  0.160311  0.163863  0.161301
All          2.668874  2.408602  2.569672  0.159328  0.163196  0.160803

In [69]:
tips.pivot_table( ['tip_pct', 'size'], index=['time', 'smoker'], columns='day',
                 aggfunc=len, margins=True)

size                          tip_pct                         
day             Fri   Sat   Sun  Thur    All     Fri   Sat   Sun  Thur    All
time   smoker                                                                
Dinner No       3.0  45.0  57.0   1.0  106.0     3.0  45.0  57.0   1.0  106.0
       Yes      9.0  42.0  19.0   NaN   70.0     9.0  42.0  19.0   NaN   70.0
Lunch  No       1.0   NaN   NaN  44.0   45.0     1.0   NaN   NaN  44.0   45.0
       Yes      6.0   NaN   NaN  17.0   23.0     6.0   NaN   NaN  17.0   23.0
All            19.0  87.0  76.0  62.0  244.0    19.0  87.0  76.0  62.0  244.0

In [70]:
tips.pivot_table('tip_pct', index=['time', 'size', 'smoker'],
                 columns='day', aggfunc='mean', fill_value=0)

day                      Fri       Sat       Sun      Thur
time   size smoker                                        
Dinner 1    No      0.000000  0.137931  0.000000  0.000000
            Yes     0.000000  0.325733  0.000000  0.000000
       2    No      0.139622  0.162705  0.168859  0.159744
            Yes     0.171297  0.148668  0.207893  0.000000
       3    No      0.000000  0.154661  0.152663  0.000000
            Yes     0.000000  0.144995  0.152660  0.000000
       4    No      0.000000  0.150096  0.148143  0.000000
            Yes     0.117750  0.124515  0.193370  0.000000
       5    No      0.000000  0.000000  0.206928  0.000000
            Yes     0.000000  0.106572  0.065660  0.000000
       6    No      0.000000  0.000000  0.103799  0.000000
Lunch  1    No      0.000000  0.000000  0.000000  0.181728
            Yes     0.223776  0.000000  0.000000  0.000000
       2    No      0.000000  0.000000  0.000000  0.166005
            Yes     0.181969  0.000000  0.000000  0.158843
       3    No      0.187735  0.000000  0.000000  0.084246
            Yes     0.000000  0.000000  0.000000  0.204952
       4    No      0.000000  0.000000  0.000000  0.138919
            Yes     0.000000  0.000000  0.000000  0.155410
       5    No      0.000000  0.000000  0.000000  0.121389
       6    No      0.000000  0.000000  0.000000  0.173706

# 交叉表：crosstab

交叉表（cross-tabulation，简称crosstab）是一种用于计算分组频率的特殊透视表。看下面的例子：

作为调查分析的一部分，我们可能想要根据国籍和用手习惯对这段数据进行统计汇总。虽然可以用pivot_table实现该功能，但是**pandas.crosstab函数**会更方便：
```
Signature: pd.crosstab(index, columns, values=None, rownames=None, colnames=None, aggfunc=None, margins=False, dropna=True, normalize=False)
```

crosstab的前两个参数可以是数组或Series，或是数组列表。就像小费数据：

In [71]:
pd.crosstab([tips.time, tips.day], tips.smoker, margins=True)

smoker        No  Yes  All
time   day                
Dinner Fri     3    9   12
       Sat    45   42   87
       Sun    57   19   76
       Thur    1    0    1
Lunch  Fri     1    6    7
       Thur   44   17   61
All          151   93  244